In [1]:
# http://docs.h2o.ai/h2o/latest-stable/h2o-docs/data-science/glm.html
# https://www.h2o.ai/blog/hyperparameter-optimization-in-h2o-grid-search-random-search-and-the-future/

In [2]:
options(warn=-1)
library(h2o)
localH2O =  h2o.init(nthreads = -1, port = 54321, max_mem_size ='16G', startH2O = TRUE)


----------------------------------------------------------------------

Your next step is to start H2O:
    > h2o.init()

For H2O package documentation, ask for help:
    > ??h2o

After starting H2O, you can use the Web UI at http://localhost:54321
For more information visit http://docs.h2o.ai

----------------------------------------------------------------------


Attaching package: 'h2o'

The following objects are masked from 'package:stats':

    cor, sd, var

The following objects are masked from 'package:base':

    %*%, %in%, &&, ||, apply, as.factor, as.numeric, colnames,
    colnames<-, ifelse, is.character, is.factor, is.numeric, log,
    log10, log1p, log2, round, signif, trunc




H2O is not running yet, starting it now...

Note:  In case of errors look at the following log files:
    C:\Users\Cameron\AppData\Local\Temp\RtmpaS2Cw6/h2o_Cameron_started_from_r.out
    C:\Users\Cameron\AppData\Local\Temp\RtmpaS2Cw6/h2o_Cameron_started_from_r.err


Starting H2O JVM and connecting: . Connection successful!

R is connected to the H2O cluster: 
    H2O cluster uptime:         5 seconds 244 milliseconds 
    H2O cluster timezone:       Australia/Sydney 
    H2O data parsing timezone:  UTC 
    H2O cluster version:        3.22.1.1 
    H2O cluster version age:    5 months and 4 days !!! 
    H2O cluster name:           H2O_started_from_R_Cameron_cnu632 
    H2O cluster total nodes:    1 
    H2O cluster total memory:   14.22 GB 
    H2O cluster total cores:    8 
    H2O cluster allowed cores:  8 
    H2O cluster healthy:        TRUE 
    H2O Connection ip:          localhost 
    H2O Connection port:        54321 
    H2O Connection proxy:       NA 
    H2O Internal Sec

## Feature Set 1

In [3]:
# load in training data
train_data <- h2o.importFile(path = './feature_set_1.csv', header=TRUE, sep=",")

  |======================================================================| 100%


In [4]:
# remove first column
train_data <- train_data[,-1]
dim(train_data)

[1] 199973   2281

In [5]:
# set names
response <- 'label'
train_data[[response]] <-as.factor(train_data[[response]])
predictors <- setdiff(names(train_data), c(response, "name"))

In [6]:
# split data into 90% training 10% validation
splits <- h2o.splitFrame(
  data = train_data, 
  ratios = 0.9,
  destination_frames = c("train.hex", "valid.hex"), seed = 123
)

train <- splits[[1]]
valid <-splits[[2]]

In [7]:
# set names for glm
y='label'
x=names(train)
x=x[-which(x==y)]

In [ ]:
# default glm with lambda 0 and alpha ignored
baseNN = h2o.deeplearning(
    training_frame = train, 
    validation_frame = valid,
    x = x, y = y,
    hidden=c(2048,2048,2048),
    epochs = 10000,
    rate=0.01,
    l1=1e-5,
    max_w2=10,
    score_validation_samples=10000,      ## sample the validation dataset (faster)
    stopping_rounds=2,
    stopping_metric="logloss", ## could be "MSE","logloss","r2"
    stopping_tolerance=0.01
)

  |                                                                      |   0%

In [ ]:
# training error 40.20, validation error 40.56
summary(baseNN)

In [ ]:
h2o.confusionMatrix(baseNN, valid=TRUE)

## Feature Set 2

In [ ]:
# load in training data
train_data <- h2o.importFile(path = './feature_set_2.csv', header=TRUE, sep=",")

In [ ]:
# remove first column
train_data <- train_data[,-1]
dim(train_data)

In [ ]:
# set names
response <- 'label'
train_data[[response]] <-as.factor(train_data[[response]])
predictors <- setdiff(names(train_data), c(response, "name"))

In [ ]:
# split data into 90% training 10% validation
splits <- h2o.splitFrame(
  data = train_data, 
  ratios = 0.9,
  destination_frames = c("train.hex", "valid.hex"), seed = 123
)

train <- splits[[1]]
valid <-splits[[2]]

In [ ]:
# set names for glm
y='label'
x=names(train)
x=x[-which(x==y)]

In [ ]:
# default glm with lambda 0 and alpha ignored
baseNN2 = h2o.deeplearning(
    training_frame = train, 
    validation_frame = valid,
    x = x, y = y,
    hidden=c(2048,2048,2048),
    epochs = 10000,
    rate=0.01,
    l1=1e-5,
    max_w2=10,
    score_validation_samples=10000,      ## sample the validation dataset (faster)
    stopping_rounds=2,
    stopping_metric="logloss", ## could be "MSE","logloss","r2"
    stopping_tolerance=0.01
)

In [ ]:
summary(baseNN2)

In [ ]:
h2o.confusionMatrix(baseNN2, valid=TRUE)

## Test Predictions

In [19]:
test_tokens_1 <- read.csv(file='./test_feature_set_1.csv', header=TRUE, sep=' ')

In [20]:
dim(test_tokens_1)

[1] 50001  2564

In [21]:
#test_data <- test_tokens_1[2:50001,]

In [22]:
pred <- h2o.predict(baseNN, test_data)
final.pred<-as.vector(pred$predict)

  |======================================================================| 100%


In [23]:
write.csv(final.pred, "./predictions_feature_set_1.csv", row.names=FALSE, quote=FALSE)

In [ ]:
test_tokens_2 <- h2o.importFile(path = './test_feature_set_2.csv', header=TRUE, sep=",")

In [ ]:
dim(test_tokens_2)

In [ ]:
#test_data <- test_tokens_2[2:50001,]

In [ ]:
pred <- h2o.predict(baseNN, test_data)
final.pred<-as.vector(pred$predict)

In [ ]:
write.csv(final.pred, "./predictions_feature_set_2.csv", row.names=FALSE, quote=FALSE)